In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
data = pd.read_excel('data.xlsx', parse_dates=['period'])
exog = pd.read_excel('exog_forecast.xlsx').sort_index().asfreq('M')

keys = ['ТБ', 'dept_id', 'Group_500', 'Валюта']
for key, group in data.groupby(by=keys):
    group.drop(keys, axis=1, inplace=True)
    group = group.set_index('period').sort_index().squeeze().asfreq('M')
    break

group.plot()
plt.show()

In [ ]:
exog.plot()

In [ ]:
from itertools import product

train, test = group[:-12], group[-12:]
params = list(product(range(1, 4), range(1, 4), range(1, 4), range(2), range(1, 2), range(1)))
MAPE_dict = {}

for param in params:
    model = SARIMAX(train, exog=exog.loc[train.index], order=(param[0], param[1], param[2]),
                    seasonal_order=(param[3], param[4], param[5], 12),
                    mle_regression=False, enforce_stationarity=False, enforce_invertibility=False).fit()
    forecast = model.forecast(len(test), exog=exog[~exog.index.isin(train.index)][:len(test)])
    MAPE = abs(forecast / test - 1).mean()
    MAPE_dict.update({MAPE: param})

min_MAPE = min(list(MAPE_dict.keys()))
param = MAPE_dict[min_MAPE]

model = SARIMAX(train, exog=exog.loc[train.index], order=(param[0], param[1], param[2]),
                    seasonal_order=(param[3], param[4], param[5], 12),
                    mle_regression=False, enforce_stationarity=False, enforce_invertibility=False).fit()
forecast = model.forecast(len(test), 
                          exog=exog[~exog.index.isin(train.index)][:len(test)])


sMAPE = (f'{min_MAPE:0.2%}')

plt.suptitle('MAPE: ' + sMAPE)
group.plot()
forecast.append(train.last('M')).sort_index().plot()
plt.show()

In [ ]:
tmp_exog = exog[~exog.index.isin(train.index)]
tmp_exog.shape

In [ ]:
param

Лин.регрессия, ошибки, Sarima(ошибки) 

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
exog.index

In [ ]:
tmp_exog = exog.reset_index()
np.array(tmp_exog.index).reshape(-1, 1)


In [ ]:
np.array(tmp_exog.index)

In [ ]:
np.array(tmp_exog.index).reshape(-1, 1).reshape(1, -1)[0]

In [ ]:
X_test[0][0]

In [ ]:
exog.index[X_test[0][0]:]

In [ ]:
Y_pred.reshape(1, -1)[0]

In [ ]:
prediction = pd.Series(Y_pred.reshape(1, -1)[0], index=exog.index[X_test[0][0]:])
prediction.plot()

In [ ]:
X = np.array(tmp_exog.index).reshape(-1, 1)
Y = np.array(exog.values).reshape(-1, 1)

X_train, X_test = X[:-12], X[-12:]
Y_train, Y_test = Y[:-12], Y[-12:]

lin_reg = LinearRegression()
lin_reg.fit(X_train, Y_train)
Y_pred = lin_reg.predict(X_test)

exog.plot()
prediction = pd.Series(Y_pred.reshape(1, -1)[0],
                       index=exog.index[X_test[0][0]:])
prediction.plot()
plt.show()